In [ ]:
import pandas as pd
from datetime import datetime,timedelta
from matplotlib import pyplot as plt      # Del modulo pyplot usaremos lo necesario para los graficos
from matplotlib import dates as mpl_dates # Del modulo dates usaremos el formato especifico para la clase date/time
import numpy as np
import requests # Para definir nuestro propio wget

In [ ]:
def wget(url):
    r = requests.get(url, allow_redirects=True)
    with open(url[url.rfind('/') + 1::], 'wb') as f:
        f.write(r.content)
wget("https://raw.githubusercontent.com/junghanss/curso_itba/master/AMZN.csv")
wget("https://raw.githubusercontent.com/junghanss/curso_itba/master/GOOG.csv")

In [ ]:
data_amzn = pd.read_csv("AMZN.csv", parse_dates=["Date"], index_col="Date")    # Leemos cada archivo csv, convirtiendo el string de la columna "Date" a formato datetime e indexando por dicha columna
data_goog = pd.read_csv("GOOG.csv", parse_dates=["Date"], index_col="Date")
u=pd.to_datetime(input("Inserta una fecha inicial: (ejemplo: 2005) "), dayfirst=True)        # Definimos la variable que almacenará el dato de la entrada de la fecha inicial del periodo a eleccion
f=pd.to_datetime(input("Inserta una fecha final: (ejemplo 2020) "), dayfirst=True)          # Se le asigna como parametro que en el ingreso respete dia primero

while u>f:          # Componemos un bucle que nos ayudara a determinar si la fecha inicial es posterior a la final, para solicitar que se reingrese el dato, sin que el programa se rompa
    print("El orden de las fechas ha sido mal ingresado, por favor intentarlo nuevamente.")     
    u=pd.to_datetime(input("Inserta una fecha inicial: "), dayfirst=True)
    f=pd.to_datetime(input("Inserta una fecha final: "), dayfirst=True)
else:               # Faltaria determinar un loop condicionando el input a un único formato, para que si el usuario ingresa un string o un dato incompatible, el programa no se rompa. Disculpas pero no llegue a codearlo.
    x=data_amzn[u:f]["Close"]       
    y=data_goog[u:f]["Close"]  

In [ ]:
crucex = []
crucey = []

for i in range(1, len(data_amzn[u:f])):         # Armamos el bucle donde determinaremos un condicional para que almacene aquellos datos de precios que son iguales o su diferencia entre dos días se cruzan
    if (data_amzn["Close"][i] == data_goog["Close"][i]) or (data_amzn["Close"][i] > data_goog["Close"][i] and data_amzn["Close"][i-1] < data_goog["Close"][i-1]) or (data_amzn["Close"][i] < data_goog["Close"][i] and data_amzn["Close"][i-1] > data_goog["Close"][i-1]):
        crucex.append(data_goog.index[i])
        crucey.append(data_goog["Close"][i])
print("Eje X: ", crucex)
print("Eje Y: ", crucey)

In [ ]:
np.savetxt("crossingdates.csv", crucex, fmt='%s')   # Con la funcion savetxt de Numpy podremos almacenar la informacion de los cruces exportandola en formato csv

In [ ]:
# EXTRA: Analisis opcional con medias moviles para el usuario
# Calculamos dos medias moviles simples (simple moving average) de periodo 20 dias (window=20)
SMA_short_AMZN = x.rolling(window=20).mean()
SMA_short_AMZN.head(20)
SMA_short_GOOG = y.rolling(window=20).mean()
SMA_short_GOOG.head(20)
# Calculamos dos medias moviles simples (simple moving average) de un periodo largo (window=80)
SMA_long_AMZN = x.rolling(window=80).mean()
SMA_long_AMZN.tail()
SMA_long_GOOG = y.rolling(window=80).mean()
SMA_long_GOOG.tail()

In [ ]:
plt.style.use('seaborn')
plt.rc('figure', figsize=(12,6))
plt.plot(x, color="#06314d", label="Amazon")
plt.plot(y, color="#91162f", label="Alphabet Inc")

MA=str(input("¿Desearías agregar medias moviles a tu grafico? (SI / NO)  :"))  # Pedimos al usuario un input para que elija si quiere o no agregar las medias moviles al grafico
if MA == "SI" or MA =="Si" or MA=="si" or MA=="sI":                            # En caso que el input tipo string coincida con esas opciones, graficara todas las SMA
    plt.plot(SMA_short_AMZN, 'g', label="SMA 20-days Amazon")
    plt.plot(SMA_long_AMZN, color="#b0aaac", label="SMA 80-days Amazon")
    plt.plot(SMA_short_GOOG, 'c', label="SMA 20-days Google")
    plt.plot(SMA_long_GOOG, color="#b49cb1", label="SMA 80-days Google")
    
plt.plot(crucex, crucey, 'k', marker='o', linestyle='none', label="Intersecciones")     # Graficamos las listas con datos de cruces para que queden marcadas dichas intersecciones
plt.gcf().autofmt_xdate()       # Llamamos a la figura (con gcf: get current figure) para auto-ajustar (autofmt) el eje X. Automaticamente las posiciones de fechas se ajustaran segun el zoom o posicion del grafico 
date_format = mpl_dates.DateFormatter('%b, %d %Y') # (b:month, d:day, Y:year) seteamos una variable con el formato elegido
plt.gca().xaxis.set_major_formatter(date_format)   # Llamamos al eje X (gca: get current axis) y le asignamos como formato a la variable anterior
plt.title("Grafico comparativo - Cotización historica", fontsize=16)
plt.xlabel("Tiempo")
plt.ylabel("Precio")
plt.legend()
plt.show()